In [2]:
import nest_asyncio
nest_asyncio.apply()

In [12]:
import asyncio
from bleak import BleakClient

# MAC-адрес BLE устройства
device_mac_address = 'DD:53:A5:1A:65:28'

# UUID сервиса и дескриптора
service_uuid = '6e400001-b5a3-f393-30a9-ffeeddc5e727'
descriptor_uuid = '6e400003-b5a3-f393-30a9-ffeeddc5e727'

async def run():
    async with BleakClient(device_mac_address) as client:
        # Поиск сервисов и дескрипторов
        services = await client.get_services()
        for service in services:
            if service.uuid == service_uuid:
                descriptors = await service.get_characteristics()
                for descriptor in descriptors:
                    print(descriptor)
                    print(descriptor.uuid)
                    if descriptor.uuid == descriptor_uuid:
                        
                        new_value = b'GD\r'
                        await client.write_gatt_descriptor(descriptor.handle, new_value)
                        print("Value written.")
                                                # Чтение значения дескриптора
                        value = await client.read_gatt_descriptor(descriptor.handle)
                        print(f"Value: {value}")
                        
                        break
                break

loop = asyncio.get_event_loop()
loop.run_until_complete(run())

BleakDeviceNotFoundError: Device with address DD:53:A5:1A:65:28 was not found.

In [1]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from bleak import BleakScanner
import pandas as pd

async def run():
    devices = await BleakScanner.discover(timeout=30) 
    return devices

loop = asyncio.get_event_loop()
result = loop.run_until_complete(run())

if result is not None:
    data = []
    for device in result:
        if device.name is not None:
            data.append({
                "name": device.name,
                "rssi": device.rssi,
                "metadata": device.metadata,
                "address": device.address,
                "device": device
            })
    if len(data) > 0:
        df = pd.DataFrame(data)
        sorted_df = df.sort_values(by=["rssi"], ascending=False)
        filtered_df = sorted_df[sorted_df["name"].str.startswith("TD_")]
        pattern = r"TD_(\d+)"
        filtered_df["number"] = filtered_df["name"].str.extract(pattern, expand=False).astype(int)
    else:
        print("No TD devices found.")
else:
    print("No devices found.")

C:\Users\Dukalis\AppData\Local\Temp\ipykernel_10336\2608258331.py:21: FutureWarning: BLEDevice.rssi is deprecated and will be removed in a future version of Bleak, use AdvertisementData.rssi instead
  "rssi": device.rssi,
C:\Users\Dukalis\AppData\Local\Temp\ipykernel_10336\2608258331.py:22: FutureWarning: BLEDevice.metadata is deprecated and will be removed in a future version of Bleak, use AdvertisementData instead
  "metadata": device.metadata,
C:\Users\Dukalis\AppData\Local\Temp\ipykernel_10336\2608258331.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["number"] = filtered_df["name"].str.extract(pattern, expand=False).astype(int)


In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import asyncio
from bleak import BleakClient

address = 'DD:53:A5:1A:65:28'
MODEL_NBR_UUID = "6e400003-b5a3-f393-30a9-ffeeddc5e727"

async def main(address):
    async with BleakClient(address) as client:
        model_number = await client.read_gatt_char(MODEL_NBR_UUID)
        print("Model Number: {0}".format("".join(map(chr, model_number))))

asyncio.run(main(address))

BleakError: Characteristic 6e400003-b5a3-f393-30a9-ffeeddc5e727 was not found!

In [2]:
def create_device_list(start, end):
    device_list = []
    for i in range(start, end+1):
        device_name = "TD_" + str(i)
        device_list.append(device_name)
    return device_list

In [3]:
req_device_list = create_device_list(383448, 387440)

In [4]:
# фильтруем датафрейм по списку имен устройств
filtered_df = df[df["name"].isin(req_device_list)]

# получаем устройства, которые есть в списке, но отсутствуют в датафрейме
missing_devices = set(req_device_list).difference(set(filtered_df["name"]))

# выводим результаты
print("Отфильтрованный датафрейм:")
print(filtered_df)
print("Отсутствующие устройства:")
print(missing_devices)

Отфильтрованный датафрейм:
          name  rssi                                           metadata  \
41   TD_386855   -62  {'uuids': ['6e400001-b5a3-f393-e0a9-ffeeddc5e7...   
227  TD_387440   -60  {'uuids': [], 'manufacturer_data': {3862: b'\x...   
252  TD_383449   -56  {'uuids': [], 'manufacturer_data': {3862: b'\x...   

               address                        device  
41   DD:53:A5:1A:65:28  DD:53:A5:1A:65:28: TD_386855  
227  C7:B3:53:CE:54:E9  C7:B3:53:CE:54:E9: TD_387440  
252  E2:F1:F3:8C:E1:45  E2:F1:F3:8C:E1:45: TD_383449  
Отсутствующие устройства:
{'TD_385485', 'TD_385817', 'TD_385681', 'TD_385875', 'TD_384134', 'TD_386888', 'TD_387303', 'TD_387218', 'TD_384733', 'TD_385409', 'TD_385735', 'TD_386893', 'TD_386953', 'TD_385387', 'TD_384149', 'TD_385717', 'TD_386266', 'TD_387401', 'TD_383842', 'TD_386529', 'TD_387413', 'TD_384839', 'TD_386296', 'TD_385426', 'TD_386713', 'TD_387322', 'TD_384591', 'TD_385342', 'TD_387171', 'TD_385115', 'TD_384622', 'TD_387133', 'TD_38681

TD_383449,  TD_383448, TD_387440, 386855

In [8]:
filtered_df

,name,rssi,metadata,address,device
41,TD_386855,-62,{'uuids': ['6e400001-b5a3-f393-e0a9-ffeeddc5e7...,DD:53:A5:1A:65:28,DD:53:A5:1A:65:28: TD_386855
227,TD_387440,-60,"{'uuids': [], 'manufacturer_data': {3862: b'\x...",C7:B3:53:CE:54:E9,C7:B3:53:CE:54:E9: TD_387440
252,TD_383449,-56,"{'uuids': [], 'manufacturer_data': {3862: b'\x...",E2:F1:F3:8C:E1:45,E2:F1:F3:8C:E1:45: TD_383449


In [10]:
filtered_df.loc[41].metadata['uuids'][0]

'6e400001-b5a3-f393-e0a9-ffeeddc5e727'

In [13]:
uuids = filtered_df.loc[146].metadata['uuids'][0].split('-')

In [14]:
uuids[-1]

'ffeeddc5e727'

In [27]:
uuid_uniq_part = uuids[-1]

In [12]:
filtered_df.loc[41].address

'DD:53:A5:1A:65:28'

In [30]:
DEVICE_ADDRESS = filtered_df.loc[110].address

In [35]:
import asyncio
from bleak import BleakClient

# UUID характеристик
RX_UUID = "6e400003-b5a3-f393-30a9-" + uuid_uniq_part
TX_UUID = "6e400002-b5a3-f393-30a9-" + uuid_uniq_part

# Адрес устройства

async def run():
    # Подключение к устройству
    async with BleakClient(DEVICE_ADDRESS, timeout=60) as client:
        # Получение характеристик
        rx_char = await client.read_gatt_char(RX_UUID)
        tx_char = await client.write_gatt_char(TX_UUID, b"Hello, World!")

        # Чтение данных
        data = await client.read_gatt_char(RX_UUID)

loop = asyncio.get_event_loop()
loop.run_until_complete(run())

BleakError: Could not get GATT services: Unreachable

In [8]:
# замените здесь на адрес МАС-устройства, которое вы хотите подключить
device = filtered_df.loc[0].device

In [9]:
device

BLEDevice(CA:17:37:73:61:98, TD_383448)

In [2]:
import asyncio
from bleak import BleakClient


async def run():
    async with BleakClient(device) as client:
        BleakClient.pair(*args, **kwargs)
loop = asyncio.get_event_loop()
loop.run_until_complete(run())

TimeoutError: 

In [12]:
import asyncio
from bleak import BleakClient

async def connect_to_device(mac_address):
    async with BleakClient(mac_address) as client:
        connected_device_info = await client.connect(timeout=10)
        print(f"Connected to: {connected_device_info}")

        # запись значения в характеристику
        value = b"HK:1:"+ str(20) + '\r'
        await client.write_gatt_char("6e400002-b5a3-f393-e0a9-" + 'ffeeddc5d9d8', value)
        await print(client.read_gatt_char("6e400003-b5a3-f393-e0a9-" + 'ffeeddc5d9d8'))

        await client.disconnect()
        print("Disconnected")


# запустите выполнение функции подключения к устройству
asyncio.run(connect_to_device(device))

BleakError: Could not get GATT services: Unreachable

In [48]:
filtered_df.loc[33].metadata['uuids']

['6e400001-b5a3-f393-e0a9-ffeeddc5d9d8']

In [13]:
filtered_df.sort_values(by='name')

,name,rssi,metadata,address,device
77,TD_383448,-62,{'uuids': ['6e400001-b5a3-f393-e0a9-ffeeddc5d9...,CA:17:37:73:61:98,CA:17:37:73:61:98: TD_383448
149,TD_383449,-60,"{'uuids': [], 'manufacturer_data': {3862: b'\x...",E2:F1:F3:8C:E1:45,E2:F1:F3:8C:E1:45: TD_383449
60,TD_386855,-70,"{'uuids': [], 'manufacturer_data': {3862: b'\x...",DD:53:A5:1A:65:28,DD:53:A5:1A:65:28: TD_386855
5,TD_387440,-54,{'uuids': ['6e400001-b5a3-f393-e0a9-ffeeddc5e9...,C7:B3:53:CE:54:E9,C7:B3:53:CE:54:E9: TD_387440


In [31]:
filtered_df.device[19]

BLEDevice(C1:90:05:44:E7:1A, TD_390506)

In [41]:
for device in filtered_df.device:
    data = device.metadata['manufacturer_data'][3862]
    
    print(f'Device:{device.name}, RSSI: {device.rssi}, address: {device.address}, uuid: {device.metadata}')
     # Расшифровка уровня топлива
    oil_level_raw = int.from_bytes(data[1:3], byteorder='little', signed=False)
    print(f"Уровень топлива: {oil_level_raw}")

    # Расшифровка напряжения встроенной батареи
    battery_voltage_raw = int.from_bytes(data[3:4], byteorder='little', signed=False)
    battery_voltage = battery_voltage_raw / 10.0
    print(f"Напряжение встроенной батареи: {battery_voltage} В")

    # Расшифровка температуры
    TD_temp_raw = int.from_bytes(data[4:5], byteorder='little', signed=True)
    print(f"Температура: {TD_temp_raw} Градус Цельсия")

    # Расшифровка версии прошивки
    TD_version_raw = int.from_bytes(data[5:6], byteorder='little', signed=False)
    print(f"Версия прошивки: {TD_version_raw}")
                    
    # Расшифровка периода (непосредственно измеренный параметр)
    TD_period_param = int.from_bytes(data[6:10], byteorder='little', signed=False)
    print(f"Период: {TD_period_param} ")

    # Расшифровка режима работы
    TD_mode_raw = int.from_bytes(data[10:11], byteorder='little', signed=False)
    TD_range = (TD_mode_raw & 0b11110000)
    if TD_range & 0x80:
        print("Диапазон датчика 4095")
    if TD_range & 0b00000000:
        print("Диапазон датчика 1024")

    TD_flags = (TD_mode_raw & 0b00001111)
    if TD_flags & 0b00000000:
        print("Обычный алгоритм расчета")
    if TD_flags & 0b00000001:
        print("Алгоритм расчета с тарировочной таблицей")

    # Расшифровка сглаживания и термокомпенсации
    TD_smoth_and_copm_raw = int.from_bytes(data[11:12], byteorder='little', signed=False)
    TD_therm_compensation = (TD_smoth_and_copm_raw & 0b11110000)
    if TD_therm_compensation & 0x80:
        print("Термокомпенсация выключена")
    if TD_therm_compensation & 0b00000000:
        print("Термокомпенсация включена")

    TD_smoothing = TD_smoth_and_copm_raw & 0x0F
    print(f'Сглаживание: {TD_smoothing}')

Device:TD_390506, RSSI: -44, address: C1:90:05:44:E7:1A, uuid: {'uuids': ['6e400001-b5a3-f393-e0a9-ffeeddc5f56a'], 'manufacturer_data': {3862: b'\x01\x01\x00$\x15\x8d\x9b\\\x00\x00\x00\x80'}}
Уровень топлива: 1
Напряжение встроенной батареи: 3.6 В
Температура: 21 Градус Цельсия
Версия прошивки: 141
Период: 23707 
Термокомпенсация выключена
Сглаживание: 0
Device:TD_390519, RSSI: -42, address: F8:56:9C:C0:3D:71, uuid: {'uuids': ['6e400001-b5a3-f393-e0a9-ffeeddc5f577'], 'manufacturer_data': {3862: b'\x01\x01\x00$\x16\x8d\xb0\\\x00\x00\x00\x80'}}
Уровень топлива: 1
Напряжение встроенной батареи: 3.6 В
Температура: 22 Градус Цельсия
Версия прошивки: 141
Период: 23728 
Термокомпенсация выключена
Сглаживание: 0
Device:TD_390513, RSSI: -24, address: F1:F6:A1:2E:67:0C, uuid: {'uuids': ['6e400001-b5a3-f393-e0a9-ffeeddc5f571'], 'manufacturer_data': {3862: b'\x01\x01\x00$\x16\x8d\xf7[\x00\x00\x00\x80'}}
Уровень топлива: 1
Напряжение встроенной батареи: 3.6 В
Температура: 22 Градус Цельсия
Версия п

C:\Users\I.V.Mitrofanov\AppData\Local\Temp\ipykernel_5232\166173740.py:2: FutureWarning: BLEDevice.metadata is deprecated and will be removed in a future version of Bleak, use AdvertisementData instead
  data = device.metadata['manufacturer_data'][3862]
C:\Users\I.V.Mitrofanov\AppData\Local\Temp\ipykernel_5232\166173740.py:4: FutureWarning: BLEDevice.rssi is deprecated and will be removed in a future version of Bleak, use AdvertisementData.rssi instead
  print(f'Device:{device.name}, RSSI: {device.rssi}, address: {device.address}, uuid: {device.metadata}')
C:\Users\I.V.Mitrofanov\AppData\Local\Temp\ipykernel_5232\166173740.py:4: FutureWarning: BLEDevice.metadata is deprecated and will be removed in a future version of Bleak, use AdvertisementData instead
  print(f'Device:{device.name}, RSSI: {device.rssi}, address: {device.address}, uuid: {device.metadata}')


In [36]:
from bleak import BleakClient

address = "C1:90:05:44:E7:1A" # MAC-адрес устройства
char_uuid = "00002a00-0000-1000-8000-00805f9b34fb" # UUID характеристики
data = b"Hello, BLE!" # данные для отправки

async with BleakClient(address) as client:
    await client.write_gatt_char(char_uuid, data)

await client.disconnect()

BleakDeviceNotFoundError: Device with address C1:90:05:44:E7:1A was not found.